In [ ]:
import tensorflow as tf

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/Fuzzy_Layer.ipynb')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from keras import backend as K
from keras.engine.topology import Layer
import numpy as np

class FuzzyLayer(Layer):

    def __init__(self, 
                 output_dim, 
                 initializer_centers=None,
                 initializer_sigmas=None, 
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        self.output_dim = output_dim
        self.initializer_centers = initializer_centers
        self.initializer_sigmas = initializer_sigmas
        super(FuzzyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_dimensions = list(input_shape)[:-1:-1]
        self.c = self.add_weight(name='c', 
                                 shape=(input_shape[-1], self.output_dim),
                                 initializer= self.initializer_centers if self.initializer_centers is not None else 'uniform',
                                 trainable=True)
        self.a = self.add_weight(name='a', 
                                 shape=(input_shape[-1], self.output_dim),
                                 initializer=self.initializer_sigmas if self.initializer_sigmas is not None else 'ones',
                                 trainable=True)
        super(FuzzyLayer, self).build(input_shape)  

    def call(self, x):
        
        aligned_x = K.repeat_elements(K.expand_dims(x, axis = -1), self.output_dim, -1)
        aligned_c = self.c
        aligned_a = self.a
        for dim in self.input_dimensions:
            aligned_c = K.repeat_elements(K.expand_dims(aligned_c, 0), dim, 0)
            aligned_a = K.repeat_elements(K.expand_dims(aligned_a, 0), dim, 0)

        xc = K.exp(-K.sum(K.square((aligned_x - aligned_c) / (2 * aligned_a)), axis=-2, keepdims=False))
        #sums = K.sum(xc,axis=-1,keepdims=True)
        #less = K.ones_like(sums) * K.epsilon()
        return xc# xc / K.maximum(sums, less)
        
    def compute_output_shape(self, input_shape):
        return tuple(input_shape[:-1]) + (self.output_dim,)

In [ ]:
from tensorflow.keras import backend
from tensorflow.keras import Layer
import numpy as np

# https://github.com/kenoma/KerasFuzzy

class DefuzzyLayer(Layer):

    def __init__(self,
                 output_dim,
                 initializer_rules_outcome=None,
                 **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        self.output_dim = output_dim
        self.initializer_rules_outcome = initializer_rules_outcome
        super(DefuzzyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_dimensions = list(input_shape)[:-1:-1]
        self.rules_outcome = self.add_weight(name='rules_outcome',
                                             shape=(input_shape[1], self.output_dim),
                                             initializer=self.initializer_rules_outcome if self.initializer_rules_outcome is not None else 'uniform',
                                             trainable=True)

        super(DefuzzyLayer, self).build(input_shape)

    def call(self, x):
        aligned_x = backend.repeat_elements(backend.expand_dims(x, axis=-1), self.output_dim, -1)
        aligned_rules_outcome = self.rules_outcome
        for dim in self.input_dimensions:
            aligned_rules_outcome = backend.repeat_elements(backend.expand_dims(aligned_rules_outcome, 0), dim, 0)

        xc = backend.sum((aligned_x * aligned_rules_outcome), axis=-2, keepdims=False)
        return xc

    def compute_output_shape(self, input_shape):
        return tuple(input_shape[:-1]) + (self.output_dim,)

In [ ]:
from tensorflow import keras

from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
from keras.utils import to_categorical
from keras.layers import Conv2D

Using TensorFlow backend.


In [ ]:
input_img = Input(shape=(256, 256, 3))

f_layer = FuzzyLayer(100)
def_layer = DefuzzyLayer(8)

layer1 = f_layer(input_img)
layer2 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(layer1)
layer3 = Dense(256)(layer2)

layer4 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(layer3)
layer5 = def_layer(layer4)

c = concatenate([layer2, layer4])

mnist_classifier = Model(input_img, c)

print("Everything ok")

Everything ok
